In [22]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
 
# =============================================================================
# 1. PREPARE DATA
# =============================================================================

df_conn = pd.read_csv('All-connections.csv', encoding='latin1')
df_trips = pd.read_csv('All-trips.csv', encoding='latin1')

print(f"Loaded {len(df_conn)} connections and {len(df_trips)} trip definitions.")
 
# Create a set of all unique trips involved in connections
unique_trips = set(df_conn['arr_trip_id']).union(set(df_conn['dep_trip_id']))
 
trip_caps = df_trips.set_index('trip_id')['cap'].to_dict()
trip_max_delay = df_trips.set_index('trip_id')['max_delay_min'].to_dict()
trip_max_advance = df_trips.set_index('trip_id')['max_advance_min'].to_dict()
 
# Algorithmic Constants
L = 300
PENALTY = 60
 
# =============================================================================
# 2. FORMULATION
# =============================================================================
 
m = gp.Model("GO_Transit_Transfer_Optimization")

# --- Define Variables ---
# d[i]: delay, a[i]: advance
d = {}
a = {}
 
for t in unique_trips:
    # set default limit to 10
    lim_adv = trip_max_advance.get(t, 10)
    lim_del = trip_max_delay.get(t, 10)
    d[t] = m.addVar(lb=0, ub=lim_del, name=f"d_{t}")
    a[t] = m.addVar(lb=0, ub=lim_adv, name=f"a_{t}")
 
# Z, W, y indexed by trip tuple (arr i, dep j, hub h)
Z = {}
W = {}
y = {}
 
for idx, row in df_conn.iterrows():
    i = row['arr_trip_id']
    j = row['dep_trip_id']
    h = row['hub_id']
    key = (i, j, h)
    Z[key] = m.addVar(vtype=GRB.BINARY, name=f"Z_{idx}")
    W[key] = m.addVar(lb=0, name=f"W_{idx}")
    # y is FREE variable to allow negative gaps
    y[key] = m.addVar(lb=-GRB.INFINITY, name=f"y_{idx}")
 
m.update()
 
# --- Objective Function ---
# Min Sum( demand * (wait + penalty-when-missed) )
obj_expr = gp.LinExpr()
 
for idx, row in df_conn.iterrows():
    key = (row['arr_trip_id'], row['dep_trip_id'], row['hub_id'])
    passengers = row['weight']
    # Z=1: Connected, Cost = wait * demand
    # Z=0: Missed, Cost = PENALTY * demand
    obj_expr += passengers * (W[key] + PENALTY * (1 - Z[key]))
 
m.setObjective(obj_expr, GRB.MINIMIZE)
 
# --- Constraints ---
 
# 1. Capacity Constraints
conn_by_dep = df_conn.groupby('dep_trip_id')
 
for j, group in conn_by_dep:
    # default capacity is 53 (single deck Bus)
    cap = trip_caps.get(j, 53)
    lhs = gp.LinExpr()
    for _, row in group.iterrows():
        key = (row['arr_trip_id'], j, row['hub_id'])
        lhs += row['weight'] * Z[key]
    m.addConstr(lhs <= cap, name=f"Cap_{j}")
 
# 2. Connection Constraints
for idx, row in df_conn.iterrows():
    i = row['arr_trip_id']
    j = row['dep_trip_id']
    h = row['hub_id']
    key = (i, j, h)
    S_A = row['arr_time_min']
    S_D = row['dep_time_min']
    M_h = row['min_wait_min']
    v_ijh = row['max_wait_min']
    # y = (Sched_Dep + d_j - a_j) - (Sched_Arr + d_i - a_i)
    m.addConstr(
        y[key] == (S_D + d[j] - a[j]) - (S_A + d[i] - a[i]),
        name=f"Def_y_{idx}"
    )
    # Lower Bound
    m.addConstr(y[key] >= M_h * Z[key] - L * (1 - Z[key]), name=f"MinWait_{idx}")
    # Upper Bound
    m.addConstr(y[key] <= v_ijh * Z[key] + L * (1 - Z[key]), name=f"MaxWait_{idx}")
    # Linearization of W = yZ: If Z=1, W = y. If Z=0, W = 0.
    m.addConstr(W[key] >= y[key] - L * (1 - Z[key]), name=f"W_LB_{idx}")
    m.addConstr(W[key] <= y[key] + L * (1 - Z[key]), name=f"W_UB_{idx}")
    m.addConstr(W[key] <= L * Z[key], name=f"W_Zero_{idx}")
 
# =============================================================================
# 3. OPTIMiZE
# =============================================================================
 
m.optimize()
 
if m.status == GRB.OPTIMAL:
    print(f"\nOptimal Objective Value: {m.objVal}")
    results = []
    kept_connections = 0
    missed_connections = 0
    for idx, row in df_conn.iterrows():
        key = (row['arr_trip_id'], row['dep_trip_id'], row['hub_id'])
        if Z[key].X > 0.5:
            kept_connections += 1
            status = 'KEPT'
            wait_time = round(y[key].X, 1)
        else:
            missed_connections += 1
            status = 'MISSED'
             # No waiting time if missed
            wait_time = 0
        results.append({
            'Arr_Trip': key[0],
            'Dep_Trip': key[1],
            'Hub': key[2],
            'Status': status,
            'Orig_Wait': row['dep_time_min'] - row['arr_time_min'],
            'New_Wait': wait_time,
            'Arr_Shift': round(d[key[0]].X - a[key[0]].X, 1),
            'Dep_Shift': round(d[key[1]].X - a[key[1]].X, 1),
            'Passengers': row['weight']
        })
    print(f"Connections Kept: {kept_connections}")
    print(f"Connections Missed: {missed_connections}")
    df_res = pd.DataFrame(results)
    if not df_res.empty:
        print("\nOptimized Connections:")
        print(df_res.head(10).to_string())
        df_res.to_csv("optimized_schedule_results.csv", index=False)
else:
    print("No feasible solution found.")
    m.computeIIS()
    m.write("model_error.ilp")
    print("Constraint conflicts written to model_error.ilp")

Loaded 116 connections and 164 trip definitions.

Starting Optimization...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 783 rows, 608 columns and 2086 nonzeros
Model fingerprint: 0xbd23f517
Variable types: 492 continuous, 116 integer (116 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e+00, 2e+04]
  Bounds range     [1e+00, 1e+01]
  RHS range        [4e+00, 2e+03]
Found heuristic solution: objective 244680.00000
Presolve removed 628 rows and 485 columns
Presolve time: 0.01s
Presolved: 155 rows, 123 columns, 419 nonzeros
Found heuristic solution: objective 134167.00000
Variable types: 85 continuous, 38 integer (38 binary)
Found heuristic solution: objective 131582.00000

Root relaxation: objective 7.404111e+04, 73 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current N